# Running Pretrained Mask R-CNN

In [2]:
!pip install opencv-contrib-python

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


## Object Detection

In [9]:
#From https://www.pyimagesearch.com/2019/06/10/keras-mask-r-cnn/

# USAGE
# python maskrcnn_predict.py --weights mask_rcnn_coco.h5 --labels coco_labels.txt --image images/30th_birthday.jpg

# import the necessary packages
from mrcnn.config import Config
from mrcnn import model as modellib
from mrcnn import visualize
import numpy as np
import colorsys
import argparse
import imutils
import random
import cv2
import os

from pathlib import Path

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-w", "--weights", required=True,
# 	help="path to Mask R-CNN model weights pre-trained on COCO")
# ap.add_argument("-l", "--labels", required=True,
# 	help="path to class labels file")
# ap.add_argument("-i", "--image", required=True,
# 	help="path to input image to apply Mask R-CNN to")
# args = vars(ap.parse_args())

args = {
    
    'labels' : str(Path(r'coco_labels.txt')),
    'images' : str(Path(r'D:\Users\Jokmenen\Documents\MEGA\Uni\Master\Thesis\SethParkResults\coco')),
    'weights' : str(Path(r'mask_rcnn_coco.h5')),
    
}


print(args['weights'])
# load the class label names from disk, one label per line
CLASS_NAMES = open(args["labels"]).read().strip().split("\n")

# generate random (but visually distinct) colors for each class label
# (thanks to Matterport Mask R-CNN for the method!)
hsv = [(i / len(CLASS_NAMES), 1, 1.0) for i in range(len(CLASS_NAMES))]
COLORS = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
random.seed(42)
random.shuffle(COLORS)

class SimpleConfig(Config):
	# give the configuration a recognizable name
	NAME = "coco_inference"

	# set the number of GPUs to use along with the number of images
	# per GPU
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

	# number of classes (we would normally add +1 for the background
	# but the background class is *already* included in the class
	# names)
	NUM_CLASSES = len(CLASS_NAMES)

# initialize the inference configuration
config = SimpleConfig()

# initialize the Mask R-CNN model for inference and then load the
# weights
print("[INFO] loading Mask R-CNN model...")
model = modellib.MaskRCNN(mode="inference", config=config,
	model_dir=os.getcwd())
model.load_weights(args["weights"], by_name=True)


Using TensorFlow backend.
W0523 19:17:15.890458 16468 deprecation_wrapper.py:118] From c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0523 19:17:15.932323 16468 deprecation_wrapper.py:118] From c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0523 19:17:15.940302 16468 deprecation_wrapper.py:118] From c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0523 19:17:15.966259 16468 deprecation_wrapper.py:118] From c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:1919: Th

mask_rcnn_coco.h5
[INFO] loading Mask R-CNN model...


W0523 19:17:17.954916 16468 deprecation_wrapper.py:118] From c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\keras\backend\tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0523 19:17:18.295007 16468 deprecation_wrapper.py:118] From c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\mask_rcnn-2.1-py3.7.egg\mrcnn\model.py:341: The name tf.log is deprecated. Please use tf.math.log instead.

W0523 19:17:18.301989 16468 deprecation.py:323] From c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\mask_rcnn-2.1-py3.7.egg\mrcnn\model.py:399: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0523 19:17:18.306977 16468 deprecation.py:506] From c:\use

In [1]:

model.keras_model.summary()
from keras.utils.vis_utils import plot_model
from IPython.display import Image 

#plot_model(model.keras_model, show_shapes=True, show_layer_names=True, to_file='model.png')
#Image('model.png')



NameError: name 'model' is not defined

In [ ]:

# load the input image, convert it from BGR to RGB channel
# ordering, and resize the image
image = cv2.imread(args["image"])
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = imutils.resize(image, width=512)

# perform a forward pass of the network to obtain the results
print("[INFO] making predictions with Mask R-CNN...")
r = model.detect([image], verbose=1)[0]

# loop over of the detected object's bounding boxes and masks
for i in range(0, r["rois"].shape[0]):
	# extract the class ID and mask for the current detection, then
	# grab the color to visualize the mask (in BGR format)
	classID = r["class_ids"][i]
	mask = r["masks"][:, :, i]
	color = COLORS[classID][::-1]

	# visualize the pixel-wise mask of the object
	image = visualize.apply_mask(image, mask, color, alpha=0.5)

# convert the image back to BGR so we can use OpenCV's drawing
# functions
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

# loop over the predicted scores and class labels
for i in range(0, len(r["scores"])):
	# extract the bounding box information, class ID, label, predicted
	# probability, and visualization color
	(startY, startX, endY, endX) = r["rois"][i]
	classID = r["class_ids"][i]
	label = CLASS_NAMES[classID]
	score = r["scores"][i]
	color = [int(c) for c in np.array(COLORS[classID]) * 255]

	# draw the bounding box, class label, and score of the object
	cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
	text = "{}: {:.3f}".format(label, score)
	y = startY - 10 if startY - 10 > 10 else startY + 10
	cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX,
		0.6, color, 2)

# show the output image
cv2.imshow("Output", image)
cv2.waitKey()

## Noun extraction

In [1]:
import spacy

nlp = spacy.load("en_core_web_md")

In [10]:
sent = "it has a long mane and tail"
ploep = nlp(sent)
ploep
#for ent in ploep.ents:
    #print(ent.text, ent.start_char, ent.end_char, ent.label_)
    
for token in ploep:
    print(token.pos_ == 'NOUN')
    
print(CLASS_NAMES)

False
False
False
False
True
False
True
['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


To test out: check if nouns from sentence are in the class labels list. Maybe I can do something with spacy labels?

In [18]:
# Explore CoCo annotations
import json
with open('annotations/captions_train2014.json') as fp:
    data = json.load(fp)
    
data['annotations']

[{'image_id': 318556,
  'id': 48,
  'caption': 'A very clean and well decorated empty bathroom'},
 {'image_id': 116100,
  'id': 67,
  'caption': 'A panoramic view of a kitchen and all of its appliances.'},
 {'image_id': 318556,
  'id': 126,
  'caption': 'A blue and white bathroom with butterfly themed wall tiles.'},
 {'image_id': 116100,
  'id': 148,
  'caption': 'A panoramic photo of a kitchen and dining room'},
 {'image_id': 379340,
  'id': 173,
  'caption': 'A graffiti-ed stop sign across the street from a red car '},
 {'image_id': 379340,
  'id': 188,
  'caption': 'A vandalized stop sign and a red beetle on the road'},
 {'image_id': 318556,
  'id': 219,
  'caption': 'A bathroom with a border of butterflies and blue paint on the walls above it.'},
 {'image_id': 318556,
  'id': 255,
  'caption': 'An angled view of a beautifully decorated bathroom.'},
 {'image_id': 134754,
  'id': 272,
  'caption': 'The two people are walking down the beach.'},
 {'image_id': 538480,
  'id': 288,
  'ca

In [20]:
with open('annotations/instances_train2014.json') as fp:
    data = json.load(fp)
    

data['info']

[{'segmentation': [[312.29,
    562.89,
    402.25,
    511.49,
    400.96,
    425.38,
    398.39,
    372.69,
    388.11,
    332.85,
    318.71,
    325.14,
    295.58,
    305.86,
    269.88,
    314.86,
    258.31,
    337.99,
    217.19,
    321.29,
    182.49,
    343.13,
    141.37,
    348.27,
    132.37,
    358.55,
    159.36,
    377.83,
    116.95,
    421.53,
    167.07,
    499.92,
    232.61,
    560.32,
    300.72,
    571.89]],
  'area': 54652.9556,
  'iscrowd': 0,
  'image_id': 480023,
  'bbox': [116.95, 305.86, 285.3, 266.03],
  'category_id': 58,
  'id': 86},
 {'segmentation': [[252.46,
    208.17,
    267.96,
    210.11,
    280.14,
    213.98,
    285.68,
    221.45,
    279.59,
    227.27,
    270.73,
    223.12,
    256.61,
    218.69,
    253.02,
    217.3,
    248.03,
    220.07,
    245.54,
    215.92,
    245.82,
    209.55,
    249.69,
    208.45]],
  'area': 421.47274999999996,
  'iscrowd': 0,
  'image_id': 50518,
  'bbox': [245.54, 208.17, 40.14, 19.1],


In [28]:
import pandas as pd

data = pd.read_csv('Post_processing.csv', index_col=0)
with open('annotations/instances_train2014.json') as fp:
    anno_dat = json.load(fp)





KeyError: 'annotations'

In [31]:
anno = pd.DataFrame(anno_dat['annotations'])
CLASS_NAMES = pd.DataFrame(anno_dat['categories'])


id    name supercategory
0   1  person        person

In [163]:
sent = data[data.label == 2].sample(1).exp.to_string()
#sent = data.loc[331049000].exp
toks = nlp(sent)
NNs = [str(tok) for tok in toks if tok.pos_ is "NOUN"]
print(toks, NNs)
CLASS_NAMES[CLASS_NAMES.name.isin(NNs)]

# #331049000
#CLASS_NAMES

44279000    they are stirring a salad ['salad']


Empty DataFrame
Columns: [id, name, supercategory]
Index: []

## With Xception 

  - Image --> xception --> I_embeddings
        Explanations --> gpt2 --> E_embeddings
  - Merge I+E --> Send to Classifier --> output if the sentence contains info-error (problem, irrelevant info is in the picture. might lead to problems.)
  data: x1= I_emb | x2 = E_emb | y = 1(contains error)/0(does not)
  
  I want to train a model to derive if the sentence contains incorrect or irrelevant information, based on image and explanation encodings
  

code adapted from: https://www.tensorflow.org/tutorials/text/image_captioning

In [8]:
import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import pandas as pd
from utils import load_image_from_dir
from pathlib import Path

IMGDIR = Path('D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\Master\\Thesis\\SethParkResults\\coco\\')
PATH = str(IMGDIR)+ "\\"
# Read the json file
# with open('annotations/captions_val2014.json', 'r') as f:
#     annotations = json.load(f)

#Store captions and image names in vectors

# all_img_name_vector = []

# for annot in annotations['annotations']:
#     image_id = annot['image_id']
#     full_coco_image_path = PATH + 'COCO_val2014_' + '%012d.jpg' % (image_id)

#     all_img_name_vector.append(full_coco_image_path)

# Shuffle captions and image_names together
# Set a random state
# # img_name_vector = shuffle(all_img_name_vector,random_state=1)

# print(img_name_vector[:50])

In [3]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [9]:
# load model for generating image embeddings

image_features_extract_model = tf.keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
)

image_features_extract_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [22]:
# Define function that is used for preprocessing the images (embeddings)

def fix_filename(qid, npy=False):
    imgID = str(qid)[:-3]
    filename = 'COCO_val2014_' + imgID.zfill(12)+'.jpg'
    if npy == True:
        filename+= '.npy'
    img_path = str(Path(IMGDIR / filename))
    return img_path

def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img_pre = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.xception.preprocess_input(img_pre)
    return img, img_path



In [11]:
#data

# #old
# arr = load_image_from_dir(363036001,IMGDIR)
# print(arr.shape, arr.dtype)
# print(arr)
# #print(load_image(363036001)[2].numpy())

# #new
# img = load_image(363036001)[0].numpy()
# print(img.shape, img.dtype)
# print(img)
# Image.fromarray(img.astype('uint8')) #Image can only show it as uint8

# #test

# load_image(363036001)

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

#tf.config.list_physical_devices('GPU')
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce GTX 970, pci bus id: 0000:01:00.0, compute capability: 5.2
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [12]:
# Generate dataset with only required  rows

# data = pd.read_csv('Post_processing.csv', index_col=0)
# data= data[(data.label == 5) | (data.label == 2)]


In [85]:
from tqdm import tqdm

# Get unique images
#encode_train = sorted(set(img_name_vector))  .astype('str')
filenames = [fix_filename(x) for x in data.index]
#print(filenames)
encode_train = sorted(set(filenames)) #the indexes of the data will be used as image id's to load the images
#print(encode_train[:50])

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
#print(image_dataset)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)



#print(list(image_dataset))
# #print(image_dataset.list_files())
for img, path in tqdm(image_dataset):
  batch_features = image_features_extract_model(img)

  #batch_features = tf.reshape(batch_features,
                              #(batch_features.shape[0], -1, batch_features.shape[3])) #Skip reshape since we didnt cut the dense layer

  for bf, p in zip(batch_features, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())
    
    


121it [00:34,  3.53it/s]


In [ ]:
# Get text embeddings

import math
from pytorch_pretrained_bert import OpenAIGPTTokenizer, OpenAIGPTModel, OpenAIGPTLMHeadModel



In [43]:
# add .npys to data and save as new file
data['img_enc'] = np.nan
for index, row in data.iterrows():
    print(index)
    data.loc[data.index == index, 'img_enc'] = pd.Series([np.load(fix_filename(int(index),True))])
    break
    
    
 
#data.loc[363036001] = 
np.load(fix_filename(363036001,True)).size  

363036001


1000

In [44]:
# Create training and validation sets using an 80-20 split
img_name_train, img_name_val = train_test_split(filenames,
                                                                    test_size=0.2,
                                                                    random_state=0)

In [45]:
filenames

['D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000363036.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000426070.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000075560.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000253810.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000093236.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000278601.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000126995.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkResults\\coco\\COCO_val2014_000000197022.jpg',
 'D:\\Users\\Jokmenen\\Documents\\MEGA\\Uni\\Master\\Thesis\\SethParkRes

In [80]:
data = pd.read_csv('Post_processing.csv', index_col=0)

data['img_enc'] = np.nan
data['qstr_enc'] = np.nan
data['exp_enc'] = np.nan

In [91]:
def load_data_pd(row):
    idx = row.name
    img_path = fix_filename(idx)+'.npy'
    exp_path = f'text_vectors\\exp\\exp_{idx}.npy'
    qstr_path = f'text_vectors\\qstr\\qstr_{idx}.npy'
    
    img_enc = np.load(img_path)
    row['img_enc'] = img_enc
    
    exp_enc = np.load(exp_path)
    row['exp_enc'] = exp_enc
    
    qstr_enc = np.load(qstr_path)
    row['qstr_enc'] = qstr_enc
    
    return row
    
#     print(img_path,'\n',exp_path,'\n',qstr_path)

data = data.apply(load_data_pd, axis=1, raw=True)

In [94]:
data.to_pickle('Post_Processing_with_Encoding.pkl')

In [97]:
test = pd.read_csv('Post_Processing_with_Encoding.csv')
test

Unnamed: 0               ans  \
0      363036001              cake   
1      426070000             couch   
2       75560012               yes   
3      253810003               yes   
4      206878001               yes   
5       93236000                no   
6      278601006               yes   
7      126995004            tennis   
8      192168002                no   
9      197022001                no   
10      62865003               yes   
11     243600002             chair   
12     302200002                no   
13       4296004           kitchen   
14     266124002             horse   
15      74603000                no   
16     212559000               hay   
17     423116003               yes   
18     260150015          pancakes   
19     247484002       living room   
20      35525002               yes   
21     118432009                no   
22     519316004           bedroom   
23     449872015               yes   
24     366421000                no   
25     147921013                no   
26      69863000                no   
27     272673007          bathroom   
28     279437000           bedroom   
29     554618006               yes   
...          ...               ...   
1938   428231000            modern   
1939   528866001               yes   
1940   188386007               yes   
1941   425077002               yes   
1942   410627002               yes   
1943   398385000                no   
1944   544607001           frisbee   
1945   171717004          bathroom   
1946   154520073       flying kite   
1947   291404001            winter   
1948   204188003               yes   
1949   309402000               yes   
1950   300124001                no   
1951    94168002                no   
1952   429290010  throwing frisbee   
1953   420110010                no   
1954   243912000               dog   
1955   493905003            tennis   
1956   412798004    playing tennis   
1957   181799001               yes   
1958   434673001            tennis   
1959   503663011          bathroom   
1960   286322004                no   
1961   460927012               yes   
1962   386707000                no   
1963   302531003               yes   
1964   102741005               yes   
1965   507135001      fire hydrant   
1966   209431000               yes   
1967   568893000                no   

                                                    exp             pred  \
0     there is a pastry base which has been adorned ...             cake   
1                                           it is a bed            couch   
2                           there are clouds in the sky              yes   
3                                          it is yellow               no   
4                       it is a big portion of a plated              yes   
5                   the train is sitting at the station               no   
6                                          it is parked               no   
7                    the man is holding a tennis racket           tennis   
8     the man is wearing a suit and a woman to weari...               no   
9     there is a slice of yellow substance with hole...               no   
10                      it is dusty and looks like dirt              yes   
11                  it is a flat and with a narrow seat            chair   
12                                  it is a city street               no   
13                  there is a stove and a refrigerator          kitchen   
14                          it has a long mane and tail            horse   
15                      there are no people in the room               no   
16             they are eating plants with a wine place            grass   
17                    the kitchen is full of appliances               no   
18           it is cut in a slices and full of toppings        breakfast   
19                               there is a entry table      living room   
20                there is a kitchen and a coffee tab

In [ ]:
def map_func(img_name, exp_name, label):
  img_tensor = np.load(img_name.decode('utf-8')+'.npy')
  exp_tensor = np.load(exp_name.decode('utf-8')+'.npy')
  return img_tensor, exp, label


Extract image features, but them into pandas.

In [105]:
print('exp:',data.exp_enc.iloc[0].shape , 'img:', data.img_enc.iloc[0].shape)


exp: (1, 12, 768) img: (1000,)


# model

In [257]:
from tensorflow import keras
from tensorflow.keras import layers

input_img = keras.Input(shape=(1000,))

input_exp = keras.Input(shape=(22,768))
flatten_exp = layers.Flatten()(input_exp)

concat = layers.concatenate(([input_img, flatten_exp]))

d = layers.Dense(64)(concat)
d = layers.Dense(64)(d)

output = layers.Dense(1, activation = 'sigmoid')(d)

model = keras.Model([input_img,input_exp],output)
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(),optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_43 (InputLayer)           [(None, 22, 768)]    0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           [(None, 1000)]       0                                            
__________________________________________________________________________________________________
flatten_16 (Flatten)            (None, 16896)        0           input_43[0][0]                   
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (None, 17896)        0           input_42[0][0]                   
                                                                 flatten_16[0][0]           

In [234]:
train['label'].to_numpy().astype('int')

array([5, 2, 5, ..., 2, 2, 5])

In [263]:
#Try training
def pad_exp(exp):
    
    
    s = exp.shape
    out = np.pad(exp, ((0,0),(0,22-s[1]),(0,0)),'constant', constant_values = 0)

    return out


data = pd.read_pickle('Post_Processing_with_Encoding.pkl')
data = data[(data.label == 2) | (data.label == 5)]
data.label[data.label == 2] = 1
data.label[data.label == 5] = 0
# data[data.label == 5, 'label'] = 0

data
data.exp_enc = data.exp_enc.apply(pad_exp)

train, test = train_test_split(data)

print(np.vstack(train['img_enc'].to_numpy()).shape, len(train))


model.fit([np.vstack(train['img_enc'].to_numpy()),
          np.vstack(train['exp_enc'].to_numpy())],
          train['label'].to_numpy().astype('bool'),
          batch_size=32,
          epochs=20)

print(model.evaluate([np.vstack(test['img_enc'].to_numpy()),
          np.vstack(test['exp_enc'].to_numpy())],
        test['label'].to_numpy().astype('bool') ),  model.metrics_names)

# model.predict([np.vstack(test['img_enc'].to_numpy()),
#           np.vstack(test['exp_enc'].to_numpy())])
          
# labels = data['label']
# img_data = data['img_enc'].to_numpy()
# exp_data = data['exp_enc'].to_numpy()

# shapes = []
# for x in train['exp_enc'].to_numpy():
#     print(x.shape)
#dataset = tf.data.Dataset.from_tensor_slices((img_data, exp_data, labels.values))



# zero = np.zeros(((2,1,22,768)))

# print(zero)
# zero[0] = train['exp_enc'].to_numpy()[0]
# zero[1] = train['exp_enc'].to_numpy()[1]

# zero

#train['exp_enc'].to_numpy()[0].reshape((1,22,768))



# print(pad_exp(train['exp_enc'].to_numpy()[0]).shape)

c:\users\jokmenen\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(1231, 1000) 1231
Epoch 1/20
39/39 [==============================] - 0s 4ms/step - loss: 0.3185 - accuracy: 0.8603
Epoch 2/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2756 - accuracy: 0.8887
Epoch 3/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2758 - accuracy: 0.8741
Epoch 4/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2688 - accuracy: 0.8838
Epoch 5/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2616 - accuracy: 0.8838
Epoch 6/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2768 - accuracy: 0.8741
Epoch 7/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2550 - accuracy: 0.8781
Epoch 8/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2620 - accuracy: 0.8814
Epoch 9/20
39/39 [==============================] - 0s 5ms/step - loss: 0.2801 - accuracy: 0.8781
Epoch 10/20
39/39 [==============================] - 0s 4ms/step - loss: 0.2477 - accuracy: 0.8838
E

In [140]:
#for feat1,feat2, targ in dataset.take(5):
#  print ('Features: {} | {}, Target: {}'.format(feat1, feat2, targ))

type(data.img_enc.iloc[1])

numpy.ndarray

ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (3,) and requested shape (3,2)